In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time


ModuleNotFoundError: No module named 'selenium'

In [2]:
driver = webdriver.Chrome()

NameError: name 'webdriver' is not defined

In [ ]:
# link 1
import csv

def url(csv_file):
    urls = []
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            urls.append(row['news_url'])
    return urls

csv_file = 'dailyhunt_tamil.csv'
url_list = url(csv_file)

In [ ]:
len(url_list)

In [ ]:
# link 2

def url(csv_file):
    urls = []
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            urls.append(row['news_url'])
    return urls

csv_file = 'dailyhunt_1.csv'
url_list_1= url(csv_file)

In [ ]:
len(url_list_1)

In [3]:
unique_url = url_list + url_list_1

NameError: name 'url_list' is not defined

In [ ]:
len(unique_url)

In [ ]:
import pandas as pd


In [ ]:
from tqdm import tqdm

In [ ]:
def get_xpath(i):
    return f"""/html/body/div[2]/div/div/div/section[1]/div/div/div/div[{i+2}]/section/section/div/section/a/div"""


In [ ]:

news_link = set(unique_url)
new_news_links = set()

for link in tqdm(unique_url, desc="links: "):

    for i in tqdm(range(17), desc="Iterations"):
        current_url = link
        try:
            driver.get(current_url)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, get_xpath(i)))
            ).click()
            new_url = driver.current_url
#             print(new_url)

            if new_url not in unique_url:
                new_news_links.add(new_url)
            if len(new_news_links) > 1000:
                break
        except Exception as e:
            print(f"An error occurred: {e}")
            break
    print(len(new_news_links))
    if len(new_news_links) > 1000:
        break

# Convert the set to a list if needed
current_links= list(new_news_links)


In [ ]:
len(url_list_1) + len(current_links)

In [ ]:
unique_2 = url_list_1 + current_links

In [ ]:
df = pd.DataFrame(unique_2, columns=['news_url'])

In [ ]:
df.to_csv('dailyhunt_2.csv', index = True)

In [ ]:
len(unique_2)

In [ ]:
# for headlines
import pandas as pd

x_path = '/html/body/div[2]/div/div/div/section[1]/div/div/div/div[1]/section/section/figcaption'
#         /html/body/div[2]/div/div/div/section[1]/div/div/div/div[1]/section/section/figcaption

df = pd.read_csv('dailyhunt_2.csv')

headlines = []

for url in tqdm(df['news_url']):
    try:
        driver.get(url)
        headline_content = driver.find_element(By.XPATH, x_path)

        # Extract the text of the headline
        headline = headline_content.text
    except:
        break

    headlines.append(headline)



# df = pd.read_csv('dailyhunt_2.csv', index_col=0)

df['headlines'] = headlines


df.to_csv('dailyhunt_2_headlines.csv', index=False)






In [ ]:
# for news_content

x_path = '/html/body/div[2]/div/div/div/section[1]/div/div/div/div[1]/section/section'

df = pd.read_csv('dailyhunt_3.csv')

contents = []

for url in tqdm (df['news_url'] , desc = 'urls'):
    try:
        driver.get(url)
        news_content = driver.find_element(By.XPATH, x_path)


        content = news_content.text
    except:
        break

    contents.append(content)



df['news_content'] = contents
df.to_csv('dailyhunt_3_content.csv', index=True)

In [ ]:
# for image url

xpath_image  = '/html/body/div[2]/div/div/div/section[1]/div/div/div/div[1]/section/section/figure/img'

df = pd.read_csv('dailyhunt_3_content.csv')

images_url = []

for url in tqdm (df['news_url'] , desc = 'urls'):
    try:
        driver.get(url)
        image_element = driver.find_element(By.XPATH, xpath_image)
        image_url = image_element.get_attribute("src")
    except:
        image_url = ''
    images_url.append(image_url)

df['image_url'] = images_url
df.to_csv('dailyhunt_4.csv', index=False)

In [ ]:
# extracting comments
xpath_1 = "/html/body/div[2]/div/div/div/section[1]/div/div/div/div[1]/section/section/div[2]/section[2]"
xpath_2 = "/html/body/div[2]/div/div/div/section[1]/div/div/div/div[1]/section/section/div[2]/section[3]"


# driver = webdriver.Chrome()


df = pd.read_csv('dailyhunt_4.csv')


comments_list = []


for url in tqdm(df['news_url'], desc = 'urls'):
    driver.get(url)
    time.sleep(5)

    try:

        comments = WebDriverWait(driver, 8).until(
            EC.presence_of_element_located((By.XPATH, xpath_1))
        ).text

    except TimeoutException:
        try:
            # If the first XPath fails, try the second XPath
            comments = WebDriverWait(driver, 8).until(
                EC.presence_of_element_located((By.XPATH, xpath_2))
            ).text

        except TimeoutException:

            comments = ""


    comments_list.append(comments)


# driver.quit()


df['reader_comments'] = comments_list


df.to_csv('dailyhunt_5.csv', index=False)


In [ ]:
/html/body/div[2]/div/div/div/section[1]/div/div/div/div[1]/section/section/div[2]/section[2]

In [ ]:
/html/body/div[2]/div/div/div/section[1]/div/div/div/div[1]/section/section/div[2]/section[2]
/html/body/div[2]/div/div/div/section[1]/div/div/div/div[1]/section/section/div[2]/section[3]

In [ ]:
print('sal')

In [ ]:
# to delete a column and rename
import pandas as pd


df = pd.read_csv('news_no_index.csv')

columns_to_delete = ['comments_new']

df.drop(columns=columns_to_delete, inplace=True)


columns_to_rename = {
    'comments_new_1': 'reader_comments'
}

df.rename(columns=columns_to_rename, inplace=True)

# Save the modified DataFrame back to a CSV file
df.to_csv('dailyhunt_tamil_modified.csv', index=False)
